In [ ]:
#| default_exp format
%load_ext autoreload
%autoreload 2

In [ ]:
# | hide
import sys
from pathlib import Path

# Insert in Path Project Directory
sys.path.insert(0, str(Path().cwd().parent))


In [ ]:
# | export
import re
from typing import Iterable, Tuple, Union, List

import pandas as pd
import numpy as np
from fastcore.utils import listify
from fastcore.xtras import Path
from geopy.distance import geodesic
from rich.progress import Progress
from pyarrow import ArrowInvalid
from dotenv import load_dotenv, find_dotenv


from extracao.constants import BW, BW_pattern, APP_ANALISE_PT, APP_ANALISE_EN

RE_BW = re.compile(BW_pattern)
MAX_DIST = 10  # Km
LIMIT_FREQ = 84812.50
load_dotenv(find_dotenv())

True

In [ ]:
# | export
def _read_df(folder: Union[str, Path], stem: str) -> pd.DataFrame:
    """Lê o dataframe formado por folder / stem.[parquet.gzip | fth | xslx]"""
    file = Path(f"{folder}/{stem}.parquet.gzip")
    try:
        df = pd.read_parquet(file)
    except (ArrowInvalid, FileNotFoundError) as e:
        raise e(f"Error when reading {file}") from e
    return df


# Formatação

> Este módulo possui funções auxiliares de formatação dos dados das várias fontes.


In [ ]:
# | export
def parse_bw(
    bw: str,  # Designação de Emissão (Largura + Classe) codificada como string
) -> Tuple[str, str]:    # Largura e Classe de Emissão
    """Parse the bandwidth string"""
    if match := re.match(RE_BW, bw):
        multiplier = BW[match[2]]
        if mantissa := match[3]:
            number = float(f"{match[1]}.{mantissa}")
        else:
            number = float(match[1])
        classe = match[4]
        return str(multiplier * number), str(classe)
    return "-1", "-1"


In [ ]:
# | export
def _filter_matlab(
    df: pd.DataFrame,  # Arquivo de Dados Base de Entrada
) -> pd.DataFrame:  # Arquivo de Dados formatado para leitura no Matlab
    """Recebe a base de dados da Anatel e formata as colunas para leitura de acordo com os requisitos do Matlab"""
    df["#Estação"] = df["Número_Estação"]
    df.loc[df.Multiplicidade != "1", "#Estação"] = (
        df.loc[df.Multiplicidade != "1", "Número_Estação"]
        + "+"
        + df.loc[df.Multiplicidade != "1", "Multiplicidade"]
    )
    cols_desc = [
        "Fonte",
        "Status",
        "Classe",
        "Entidade",
        "Fistel",
        "#Estação",
        "Município_IBGE",
        "UF",
    ]
    df.loc[:, cols_desc].fillna("NI", inplace=True)

    df["Descrição"] = (
        "["
        + df.Fonte
        + "] "
        + df.Status
        + ", "
        + df.Classe
        + ", "
        + df.Entidade.str.title()
        + " ("
        + df.Fistel
        + ", "
        + df["#Estação"]
        + "), "
        + df.Município_IBGE
        + "/"
        + df.UF
    )

    bad_coords = df.Coords_Valida_IBGE == "0"

    df.loc[bad_coords, "Descrição"] = df.loc[bad_coords, "Descrição"] + "*"

    df.loc[bad_coords, ["Latitude", "Longitude"]] = df.loc[
        bad_coords, ["Latitude_IBGE", "Longitude_IBGE"]
    ].values

    df = df.loc[:, APP_ANALISE_PT]
    df.columns = APP_ANALISE_EN
    return df

In [ ]:
# | export
def _format_matlab(
    df: pd.DataFrame,  # Arquivo de Dados Base de Entrada
) -> pd.DataFrame:  # Arquivo de Dados formatado para leitura no Matlab
    """Formata o arquivo final de dados para o formato esperado pela aplicação em Matlab"""
    df = df.astype("string")
    df.loc[len(df), :] = [
        "-1",
        "-15.7801",
        "-47.9292",
        "[TEMP] L, FX, Estação do SMP licenciada (cadastro temporário)",
        "10",
        "999999999",
        "NI",
        "-1",
    ]  # Paliativo...
    for c in ["Latitude", "Longitude"]:
        df[c] = df[c].fillna(-1).astype("float32")
    df["Frequency"] = df["Frequency"].astype("float64")
    df.loc[df.Service.isin(["", "-1"]), "Service"] = pd.NA
    df["Service"] = df.Service.fillna("-1").astype("int16")
    df.loc[df.Station.isin(["", "-1"]), "Station"] = pd.NA
    df["Station"] = df.Station.fillna("-1").astype("int32")
    df.loc[df.BW.isin(["", "-1"]), "BW"] = pd.NA
    df["BW"] = df["BW"].astype("float32").fillna(-1)
    df.loc[df["Class"].isin(["", "-1"]), "Class"] = pd.NA
    df["Class"] = df.Class.fillna("NI").astype("category")
    df = df[df.Frequency <= LIMIT_FREQ]
    df.sort_values(
        by=["Frequency", "Latitude", "Longitude", "Description"], inplace=True
    )
    unique_columns = df.columns.tolist().remove("Description")
    df = df.drop_duplicates(subset=unique_columns, keep="last").reset_index(drop=True)
    df["Id"] = [f"#{i+1}" for i in df.index]
    df["Id"] = df.Id.astype("string")
    df.loc[df.Description == "", "Description"] = pd.NA
    df["Description"] = df["Description"].astype("string").fillna("NI")
    return df[["Id"] + list(APP_ANALISE_EN)]

### Mesclagem
Função auxiliar para mesclar registros que são iguais das diversas bases, i.e. estão a uma distância menor que `MAX_DIST` e verificar a validade da mesclagem

In [ ]:
# | export
def get_km_distance(row):
    return geodesic((row[0], row[1]), (row[2], row[3])).km


def merge_on_frequency(df_left, df_right):
    """Mescla os registros dos DataFrames `df_left` e `df_right` que estão a uma distância menor que MAX_DIST"""
    df1 = df_left.copy().reset_index(drop=True)
    df2 = df_right.copy().reset_index(drop=True)
    columns = ["Frequency", "Latitude", "Longitude"]
    for c in columns:
        df1[c] = df1[c].astype("float")
        df2[c] = df2[c].astype("float")
    df1.sort_values(columns, inplace=True)
    df2.sort_values(columns, inplace=True)
    with Progress(transient=True, refresh_per_second=2) as progress:
        task_left = progress.add_task(
            "[red]Iterando Tabela Principal...", total=len(df1)
        )
        for left in df1.itertuples():
            for right in df2[np.isclose(df2.Frequency, left.Frequency)].itertuples():
                if (
                    geodesic(
                        (left.Latitude, left.Longitude),
                        (right.Latitude, right.Longitude),
                    ).km
                    <= MAX_DIST
                ):
                    df1.loc[
                        left.Index, "Description"
                    ] = f"{left.Description} | {right.Description}"
                    df2 = df2.drop(right.Index)
                    break
            progress.update(
                task_left,
                advance=1,
                description=f"[green] Comparando Frequências {left.Frequency}MHz",
            )
        return pd.concat([df1, df2], ignore_index=True).astype('string')


In [ ]:
folder = Path.cwd().parent / "dados"